# The XL algorithm (solutions)
This notebook is part of the exercise on XL/FXL.
Here we made the outline for the implementation with some details missing.
Fill in the missing functionality and test your implementation.
See the accompanying markdown file `FXL-exercise.md` for more details.

### Exercise: Implement the functions in the cells below

In [ ]:
# Should return all monomials of R of degree less or equal than D
# It might be wise to sort the monomials to your liking
def monomials_of_degree_le(R, D):
    return sum([R.monomials_of_degree(i) for i in range(D+1)], [])[::-1]

In [ ]:

# Should return all products f * m, such that deg(f * m) <= D, with f a system polynomial and m a monomial of R
def augmented_polynomials(R, fs, D):
    return sorted([f * m for f in fs for m in monomials_of_degree_le(R, D - f.degree())])[::-1]

In [ ]:
# Given the monomials by which the columns are indexed and the augmented polynomials by which the rows are indexed
# Return the Macaulay matrix entries as a two-dimensional array
def macaulay_matrix_entries(F, monomials, aug_polys):
    return [[f.monomial_coefficient(mon) for mon in monomials] for f in aug_polys]

In [ ]:

# Retrieve a solution from a right kernel vector of the Macaulay matrix
# Hint: this is dependent on the order of the monomials (i.e. the columns)
def kernel_vector_to_solution(n, v):
    return v[-(n+1):-1]/v[-1]

### As part of the XL algorithm

In [ ]:
# Note that n, m are implied by R, fs

# Build the Macaulay matrix for fs in degree D
def generate_Macaulay_matrix(R, fs, D):
    F = R.base_ring()

    monomials = monomials_of_degree_le(R, D)
    aug_polys = augmented_polynomials(R, fs, D)

    entries = macaulay_matrix_entries(F, monomials, aug_polys)

    Mac_D = matrix(F, len(aug_polys), len(monomials), entries)

    return Mac_D

In [ ]:
# Run the XL algorithm in degree D for such a system
def XL(R, fs, D):
    mac_D = generate_Macaulay_matrix(R, fs, D)

    K = mac_D.right_kernel()

    # Check that we get a unique solution before proceeding, this is just for simplicity
    assert K.dimension() == 1 

    solution = kernel_vector_to_solution(len(R.gens()), K.basis()[0])

    return solution

### Time to test our implementation

In [ ]:
# This generates a random MQ problem in the Ring R = F[x1, ..., xn] of m quadratic equations.
# Note the field F and number of variables n are already defined by R.
def generate_MQ_problem(R, m):
    F = R.base_ring()
    n = len(R.gens())
    quadratic_monomials = R.monomials_of_degree(2)
    linear_monomials = R.monomials_of_degree(1)
    monomials = quadratic_monomials + linear_monomials

    # Generate random polynomials
    fs = [sum(F.random_element() * mon for mon in monomials) for _ in range(m)]

    # Generate random solution
    solution = [F.random_element() for _ in range(n)]

    # Change the constant to match the solution
    fs = [f - f(solution) for f in fs]

    return fs, solution

In [ ]:
q = 7
n = 12
m = 30

F = GF(q)
R = PolynomialRing(F, n, 'x', order="degrevlex")

In [ ]:
fs, solution = generate_MQ_problem(R, m)
sol = XL(R, fs, 4)

# Check correctness of the solution
assert all(f(list(sol)) == 0 for f in fs)

print("Planted solution:", solution)
print("Found   solution:", sol)

## Solutions FXL

We need to redefine XL in order to allow for systems without solutions

In [ ]:

def XL(R, fs, D):
    mac_D = generate_Macaulay_matrix(R, fs, D)
    K = mac_D.right_kernel()
    if K.dimension() < 1:
        return False
    
    return kernel_vector_to_solution(len(R.gens()), K.basis()[0])

In [ ]:
def FXL(R, fs, D, k):
    n = len(R.gens())
    F = R.base_ring()
    S = PolynomialRing(F, n-k, 'y', order='degrevlex')

    for g in VectorSpace(F, k):
        phi = R.hom(list(S.gens()) + list(g))
        fs_S = [phi(f) for f in fs]
        solution = XL(S, fs_S, D)
        if solution != False:
            return list(solution) + list(g)
    

In [ ]:
fs, solution = generate_MQ_problem(R, m)
sol = FXL(R, fs, 3, 2)

# Check correctness of the solution
assert all(f(list(sol)) == 0 for f in fs)

print("Planted solution:", solution)
print("Found   solution:", sol)